In [1]:
from dataclasses import dataclass, field
from math import prod, lcm
from operator import attrgetter
from typing import Callable, ClassVar
from __future__ import annotations

@dataclass
class Monkey:
    order: int
    items: list[int]
    operation: Callable[[int], int]
    divisor: int
    if_true: int
    if_false: int
    inspects: int = 0
    monkeys: ClassVar[dict[int, Monkey]] = {}
    
    def __post_init__(self):
        self.order = int(self.order)
        self.items = list(map(int, self.items.split(", ")))
        self.operation = eval(f"lambda old: {self.operation}")
        self.divisor = int(self.divisor)
        self.if_true = int(self.if_true)
        self.if_false = int(self.if_false)
        Monkey.monkeys[self.order] = self
        Monkey._lcm = 0
        
    def run(self, with_divide: bool=True, with_lcm: bool=False):
        while self.items and (old := self.items.pop(0)):
            self.inspects += 1
            new = self.operation(old)
            if with_divide:
                new //= 3
            if with_lcm:
                new %= Monkey.lcm
            index = self.if_false if new % self.divisor else self.if_true
            Monkey.monkeys[index].items.append(new)
            
    @classmethod
    @property
    def lcm(cls):
        if not cls._lcm:
            cls._lcm = lcm(*(monkey.divisor for monkey in cls.monkeys.values()))
        return cls._lcm

In [2]:
import re

regexes = (
    re.compile(r"Monkey (?P<order>\d+):"),
    re.compile(r"Starting items: (?P<items>((\d+)[, ]*)+)"),
    re.compile(r"Operation: new = (?P<operation>.*)"),
    re.compile(r"Test: divisible by (?P<divisor>\d+)"),
    re.compile(f"If true: throw to monkey (?P<if_true>\d+)"),
    re.compile(f"If false: throw to monkey (?P<if_false>\d+)"),
)

all_data = []
with open("Day11.txt") as file:
    data = {}
    for line in file:
        if not line.strip():
            all_data.append(data.copy())
            data.clear()
        for regex in regexes:
            if match := regex.match(line.strip()):
                for key, value in match.groupdict().items():
                    data[key] = value
    if data:
        all_data.append(data.copy())

sort_key = attrgetter("inspects")

In [3]:
monkeys = [Monkey(**data) for data in all_data]
for i in range(20):
    for monkey in monkeys:
        monkey.run(with_divide=True)
prod(monkey.inspects for monkey in sorted(monkeys, key=sort_key)[-2:])

101436

In [4]:
monkeys = [Monkey(**data) for data in all_data]
for i in range(10000):
    for monkey in monkeys:
        monkey.run(with_divide=False, with_lcm=True)
prod(monkey.inspects for monkey in sorted(monkeys, key=sort_key)[-2:])

19754471646